# SNAPISTA Development Notebook

This notebook attempts to implement the SNAPPY InSAR Processing via the SNAP GPT Python API called SNAPISTA.

Appears to have some issues for some operators. Is also quite slow.

In [1]:
import snapista
from snapista import Operator, OperatorParams, Graph
from pathlib import Path

INFO: org.esa.snap.core.gpf.operators.tooladapter.ToolAdapterIO: Initializing external tool adapters
INFO: org.esa.s2tbx.dataio.gdal.GDALVersion: Incompatible GDAL 3.6.2 found on system. Internal GDAL 3.0.0 from distribution will be used.
INFO: org.esa.s2tbx.dataio.gdal.GDALVersion: Internal GDAL 3.0.0 set to be used by SNAP.
INFO: org.esa.snap.core.util.EngineVersionCheckActivator: Please check regularly for new updates for the best SNAP experience.
Currently installed 8.0, available is 9.0.0.
Please visit http://step.esa.int



In [3]:
########################
# USER CONFIG
# Specify the Input Filepaths
work_dir = Path('/data/S1_InSAR_Juukan/')

# NOTE: READ THE FILES IN CHRONOLOGICAL ORDER (OLDEST FIRST)
filepath_1 = Path(work_dir, 'data_raw', 'S1B_IW_SLC__1SDV_20200516T213951_20200516T214019_021612_029081_ACBD.zip')
filepath_2 = Path(work_dir, 'data_raw', 'S1B_IW_SLC__1SDV_20200528T213952_20200528T214019_021787_0295B1_17C1.zip')


In [4]:
########################
# DO NOT MODIFY
# Constructing Additional Directories
work_dir.mkdir(exist_ok=True, parents=True)

temp_dir = Path(work_dir, "Temp_SNAPISTA")
temp_dir.mkdir(exist_ok=True, parents=True)

# Specifying Output Filepaths
pre_snaphu_path = Path(
    temp_dir,
    f"{filepath_1.stem[:25]}_{filepath_2.stem[33:41]}_ELEVATION_split_Orb_Stack_esd_ifg_deb_flt.dim",
)
snaphu_export_path = Path(temp_dir, f'{filepath_1.stem}_Orb_Stack_Ifg_Deb_Flt')
processed_product_path = Path(work_dir, f"{pre_snaphu_path.stem}_unw_dem_tc.dim")

# Pre-SNAPHU Processing

In [5]:
# Process Read 1
read_1 = Operator("Read", file=str(filepath_1))

topsar_split_1 = Operator(
    "TOPSAR-Split",
    subswath="IW2",
    selectedPolarisations="VV",
    firstBurstIndex="3",
    lastBurstIndex="5",
)

orbit_1 = Operator("Apply-Orbit-File", orbitType="Sentinel Precise (Auto Download)")


In [6]:
# Process Read 2
read_2 = Operator("Read", file=str(filepath_2))

topsar_split_2 = Operator(
    "TOPSAR-Split",
    subswath="IW2",
    selectedPolarisations="VV",
    firstBurstIndex="1",
    lastBurstIndex="3",
)

orbit_2 = Operator("Apply-Orbit-File", orbitType="Sentinel Precise (Auto Download)")


In [7]:
# Coregistration
back_geocoding = Operator("Back-Geocoding", demName="SRTM 1Sec HGT")

enhanced_spectral_diversity = Operator("Enhanced-Spectral-Diversity")

# Interferogram
interferogram = Operator(
    "Interferogram",
    subtractFlatEarthPhase="true",
    subtractTopographicPhase="false",
    includeCoherence="true",
    demName="SRTM 1Sec HGT",
)

# TOPS Deburst
tops_deburst = Operator("TOPSAR-Deburst")

# Goldstein Filtering
goldstein_filtering = Operator("GoldsteinPhaseFiltering")

# Multilooking
multilook = Operator("Multilook")

In [8]:
# SNAPHU EXPORT
snaphu_export = Operator(
    "SnaphuExport",
    targetFolder=str(temp_dir),
    statCostMode="TOPO",
    initMethod="MCF",
    numberOfProcessors="8",
)

In [9]:
# Write Pre-SNAPHU File
write_presnaphu = Operator("Write", file=str(pre_snaphu_path), formatName="BEAM-DIMAP")

In [10]:
g = Graph()

# Process 1st Image
g.add_node(operator=read_1, node_id="Read(1)")

g.add_node(operator=topsar_split_1, node_id="TOPSAR-Split(1)", source="Read(1)")

g.add_node(operator=orbit_1, node_id="Apply-Orbit-File(1)", source="TOPSAR-Split(1)")

# Process 2nd Image
g.add_node(operator=read_2, node_id="Read(2)")

g.add_node(operator=topsar_split_2, node_id="TOPSAR-Split(2)", source="Read(2)")

g.add_node(operator=orbit_2, node_id="Apply-Orbit-File(2)", source="TOPSAR-Split(2)")

# Coregistration
g.add_node(
    operator=back_geocoding,
    node_id="Back-Geocoding",
    source=["Apply-Orbit-File(1)", "Apply-Orbit-File(2)"],
)

g.add_node(
    operator=enhanced_spectral_diversity,
    node_id="Enhanced-Spectral-Diversity",
    source="Back-Geocoding",
)

# Interferogram Formation
g.add_node(
    operator=interferogram,
    node_id="Interferogram",
    source="Enhanced-Spectral-Diversity",
)

# TOPS Deburst
g.add_node(operator=tops_deburst, node_id="TOPSAR-Deburst", source="Interferogram")

# Goldstein Filtering
g.add_node(
    operator=goldstein_filtering,
    node_id="GoldsteinPhaseFiltering",
    source="TOPSAR-Deburst",
)

# Multilooking
# g.add_node(operator=multilook, node_id="Multilook", source="GoldsteinPhaseFiltering")

# SNAPHU Export
g.add_node(operator=snaphu_export, node_id="SnaphuExport", source="GoldsteinPhaseFiltering")

# Write Pre-SNAPHU-Processed Product
g.add_node(operator=write_presnaphu, node_id="Write", source="GoldsteinPhaseFiltering")


Issue with Multilook not selecting ALL S1 Bands, only the first one.

No easy way to extract all the band names with SNAPISTA.

https://forum.step.esa.int/t/empty-sourcebands-does-not-select-all-s1-bands-in-snap-8-graphs/33961

In [11]:
g.view()


<graph>
  <version>1.0</version>
  <node id="Read(1)">
    <operator>Read</operator>
    <sources/>
    <parameters class="com.bc.ceres.binding.dom.XppDomElement">
      <bandNames/>
      <copyMetadata>true</copyMetadata>
      <file>/data/S1_InSAR_Juukan/data_raw/S1B_IW_SLC__1SDV_20200516T213951_20200516T214019_021612_029081_ACBD.zip</file>
      <formatName/>
      <geometryRegion/>
      <maskNames/>
      <pixelRegion/>
    </parameters>
  </node>
  <node id="TOPSAR-Split(1)">
    <operator>TOPSAR-Split</operator>
    <sources>
      <sourceProduct refid="Read(1)"/>
    </sources>
    <parameters class="com.bc.ceres.binding.dom.XppDomElement">
      <firstBurstIndex>3</firstBurstIndex>
      <lastBurstIndex>5</lastBurstIndex>
      <selectedPolarisations>VV</selectedPolarisations>
      <subswath>IW2</subswath>
      <wktAoi/>
    </parameters>
  </node>
  <node id="Apply-Orbit-File(1)">
    <operator>Apply-Orbit-File</operator>
    <sources>
      <sourceProduct refid="TOPSAR-Spl

In [12]:
g.run()

Processing the graph
Executing processing graph
INFO: org.esa.snap.core.gpf.operators.tooladapter.ToolAdapterIO: Initializing external tool adapters
....10%....20%....30%....40%....50%....60%....70%....80%....90% done.
INFO: org.esa.s2tbx.dataio.gdal.GDALVersion: Incompatible GDAL 3.6.2 found on system. Internal GDAL 3.0.0 from distribution will be used.
INFO: org.esa.s2tbx.dataio.gdal.GDALVersion: Internal GDAL 3.0.0 set to be used by SNAP.
INFO: org.esa.snap.core.util.EngineVersionCheckActivator: Please check regularly for new updates for the best SNAP experience.
Currently installed 8.0, available is 9.0.0.
Please visit http://step.esa.int

INFO: org.esa.s2tbx.dataio.gdal.GDALVersion: Internal GDAL 3.0.0 set to be used by SNAP.
INFO: org.hsqldb.persist.Logger: dataFileCache open start
INFO: org.esa.s1tbx.sentinel1.gpf.SpectralDiversityOp: Shifts written to file: /home/ubuntu/mambaforge/envs/snap-8/snap/.snap/var/log/IW2_range_shifts.json
INFO: org.esa.s1tbx.sentinel1.gpf.SpectralDiv

0

# SNAPHU Unwrapping

In [5]:
import sys
sys.path.append('..')
from src.processing_utils import snaphu_unwrapping

snaphu_unwrapping(filepath=snaphu_export_path)


snaphu v2.0.5
27 parameters input from file snaphu.conf (84 lines total)


Logging run-time parameters to file snaphu.log
Creating temporary directory snaphu_tiles_248900
Unwrapping tile at row 0, column 0 (pid 248901)
Unwrapping tile at row 0, column 1 (pid 248902)
Unwrapping tile at row 0, column 2 (pid 248906)
Unwrapping tile at row 0, column 3 (pid 248908)
Unwrapping tile at row 0, column 4 (pid 248909)
Unwrapping tile at row 0, column 5 (pid 248910)
Unwrapping tile at row 0, column 6 (pid 248911)


# Post SNAPHU Processing

In [ ]:
g2 = Graph()

In [28]:
filepath = Path(
    "/scratch/InSAR_data/",
    "S1B_IW_SLC__1SDV_20190702_20190708_split_Orb_Stack_esd_ifg_deb_flt_ML.dim",
)
snaphu_export_path = Path("/scratch/InSAR_data/SNAPHU_Temp")


In [29]:
read = Operator(
    "Read", file=str(filepath), formatName="BEAM-DIMAP", copyMetadata="true"
)

snaphu_export = Operator(
    "SnaphuExport",
    targetFolder=str(snaphu_export_path),
    statCostMode="TOPO",
    initMethod="MCF",
    numberOfProcessors="8",
)


In [30]:
g_new = Graph()

g_new.add_node(operator=read, node_id="Read")

g_new.add_node(operator=snaphu_export, node_id="SNAPHU-Export", source="Read")

g_new.view()


<graph>
  <version>1.0</version>
  <node id="Read">
    <operator>Read</operator>
    <sources/>
    <parameters class="com.bc.ceres.binding.dom.XppDomElement">
      <bandNames/>
      <copyMetadata>true</copyMetadata>
      <file>/scratch/InSAR_data/S1B_IW_SLC__1SDV_20190702_20190708_split_Orb_Stack_esd_ifg_deb_flt_ML.dim</file>
      <formatName>BEAM-DIMAP</formatName>
      <geometryRegion/>
      <maskNames/>
      <pixelRegion/>
    </parameters>
  </node>
  <node id="SNAPHU-Export">
    <operator>SnaphuExport</operator>
    <sources>
      <sourceProduct refid="Read"/>
    </sources>
    <parameters class="com.bc.ceres.binding.dom.XppDomElement">
      <colOverlap>200</colOverlap>
      <initMethod>MCF</initMethod>
      <numberOfProcessors>8</numberOfProcessors>
      <numberOfTileCols>10</numberOfTileCols>
      <numberOfTileRows>10</numberOfTileRows>
      <rowOverlap>200</rowOverlap>
      <statCostMode>TOPO</statCostMode>
      <targetFolder>/scratch/InSAR_data/SNAPHU_Temp<

In [31]:
g_new.run()


Processing the graph
Executing processing graph
INFO: org.esa.snap.core.gpf.operators.tooladapter.ToolAdapterIO: Initializing external tool adapters


KeyboardInterrupt: 